In [1]:
import pandas as pd
from pathlib import Path

In [2]:
%run ./20-siamese-network/model.py

In [3]:
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor
import torch
import torch.nn.functional as F
import torch.nn as nn

In [4]:
DATA_PATH = Path("../data")
COMP_DATA_PATH = Path("../data")
IMAGE_PATH = Path("../dataset")
BATCH_SIZE = 4
IMG_SIZE = (1024, 1024)
EPOCHS = 5
TRAIN_SPLIT_FILTERED_PATH = DATA_PATH / "train_split_filtered.csv"
VAL_SPLIT_FILTERED_PATH = DATA_PATH / "val_split_filtered.csv"
train_split_filtered = pd.read_csv(TRAIN_SPLIT_FILTERED_PATH)
val_split_filtered = pd.read_csv(VAL_SPLIT_FILTERED_PATH)
train_transforms = transforms.Compose(
    [
        transforms.Resize((IMG_SIZE)),
        transforms.ToTensor()
    ]
)
val_transforms = transforms.Compose(
    [
        transforms.Resize((IMG_SIZE)),
        transforms.ToTensor()
    ]
)
train_dataset = SiameseNetworkDataset(train_split_filtered[:20], transform=train_transforms)
val_dataset = SiameseNetworkDataset(val_split_filtered[:20], transform=val_transforms)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [5]:
%run ./21-resnet-siamese/model.py

In [6]:
model = SiameseResNet()

In [7]:
from pytorch_lightning.loggers import WandbLogger

In [8]:
wandb_logger = WandbLogger(name="resnet-siamese-test", project="csc_hackathon_lun")

In [9]:
trainer = pl.Trainer(
    limit_train_batches=100, 
    max_epochs=5,
    accelerator="cpu",
    log_every_n_steps=1,
    callbacks=[EarlyStopping(monitor="val_loss", mode="min")],
    logger=wandb_logger
    )
trainer.fit(model=model, 
            train_dataloaders=train_dataloader,
            val_dataloaders=val_dataloader)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/nikiandr/anaconda3/envs/torch/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:176: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
/home/nikiandr/anaconda3/envs/torch/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(

  | Name   

Sanity Checking: 0it [00:00, ?it/s]

/home/nikiandr/anaconda3/envs/torch/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/nikiandr/anaconda3/envs/torch/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/home/nikiandr/anaconda3/envs/torch/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:52: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
